In [1]:
import numpy as np 
import pandas as pd 
import itertools

import pickle

import matplotlib.pyplot as plt 
import matplotlib.ticker as mtick

%matplotlib inline  

# two functions

In [2]:
def lin_func( x, sigma_sq, n ):
    
    noise = np.random.normal(0, np.sqrt( sigma_sq ) ) 
        
    return np.dot ( np.array( [1]* n ), x ) 
# + noise 

def nonlin_func( x, sigma_sq, n ): 
    
    noise = np.random.normal(0, np.sqrt( sigma_sq ) )
        
    x = np.sum( np.sin(x - np.array( [1]* n ) ) )  
    
    return x 
# + noise


def concave_func( x, sigma_sq, n ): 
    
    noise = np.random.normal(0, np.sqrt( sigma_sq ) )
        
    x = - np.sum( x**2 ) 
    
    return x 
# + noise 

In [21]:
SAMPLE_SIZE = 100

# estimation over sphere


In [42]:
def exp_map(v, n ): 
        
    return np.array( [1] + [0] * (n-1) ) * np.cos( np.linalg.norm( v ) ) \
                    + np.sin( np.linalg.norm( v ) ) * v / np.linalg.norm( v ) 

In [43]:
def gaussian_est( func, true_grad, n, mu = 0.05 , m = 100 , noise_var = 0.0001 ):

    avg_est = 0 
    est_var = 0
    
    for _ in range(m):

        v = np.random.multivariate_normal(np.zeros(n-1), np.identity(n-1) )
        v = np.array( [0] + list(v) ) 
                
        f1 = func( exp_map( mu * v, n), noise_var, n ) 
        f2 = func( exp_map( -mu * v, n), noise_var, n )

        grad_est = (f1 - f2) / ( 2 * mu ) * v 

        avg_est = avg_est + grad_est  
        
        est_var = est_var + (np.linalg.norm( grad_est )  ) ** 4

    avg_est = avg_est / m 
    est_var = est_var / m
    
    return avg_est, est_var

In [44]:
def sphere_est( func, true_grad, n, alpha, mu = 0.05 , m = 100 , noise_var = 0.0001 ):

    avg_est = 0
    est_var = 0

    for _ in range(m):

        v = np.random.multivariate_normal(np.zeros(n-1), np.identity(n-1) )
        v = v/np.linalg.norm(v) * alpha 
        v = np.array( [0] + list(v) ) 
        
        f1 = func( exp_map(mu * v, n), noise_var, n ) 
        f2 = func( exp_map(-mu * v, n), noise_var, n ) 

        grad_est = n * (f1 - f2) / ( 2 * mu * alpha ** 2 ) * v 

        avg_est = avg_est + grad_est 
        
        est_var = est_var + (np.linalg.norm( grad_est )  ) ** 4

    avg_est = avg_est / m 
    est_var = est_var / m
    
    return avg_est, est_var

In [45]:
def sphere_res(n, alpha, mu, true_grad, f, N = 100 ): 

    g_errors = [] ; g_vars = []

    for _ in range(N): 
        
        avg_est, est_var = gaussian_est(f, true_grad, n = n, m = SAMPLE_SIZE, mu = mu )
    
        g_errors.append( np.linalg.norm( avg_est - true_grad ) ) 
        
        g_vars.append(est_var) 
    
    s_errors = [] ; s_vars = []

    true_grad_lin = np.array( [0] + [1] * (n-1) )

    for _ in range(N): 
        
        avg_est, est_var = sphere_est(f, true_grad, n = n, alpha = alpha, m = SAMPLE_SIZE, mu = mu ) 
    
        s_errors.append( np.linalg.norm( avg_est - true_grad ) ) 
        
        s_vars.append(est_var)
    
    return g_errors, s_errors, g_vars, s_vars

In [46]:
ns = [10,20,30] 
alphas = [1]  
betas = [1./2]  
mus = [0.01,0.001] 

n_alpha_mu_beta = list( itertools.product( ns, alphas, mus, betas ) ) 

res = [] 

for (n, alpha, mu, beta) in n_alpha_mu_beta: 
    
    print(n,alpha,mu,beta) 
    
    true_grad_lin = np.array( [0] + [1] * n ) 
            
    g_errors, s_errors, g_vars, s_vars = sphere_res( n+1, np.sqrt(n), mu, true_grad_lin, lin_func )  
    
    pickle.dump( (g_errors, s_errors, g_vars, s_vars), open( './sphere_lin/n={0}_mu={1}'.format(n , mu) , 'wb' ) ) 

10 1 0.01 0.5
10 1 0.001 0.5
20 1 0.01 0.5
20 1 0.001 0.5
30 1 0.01 0.5
30 1 0.001 0.5


In [47]:
ns = [10,20,30] 
alphas = [1]  
betas = [1./2]  
mus = [0.01,0.001] 

n_alpha_mu_beta = list( itertools.product( ns, alphas, mus, betas ) ) 

res = [] 

for (n, alpha, mu, beta) in n_alpha_mu_beta: 
    
    print(n,alpha,mu,beta)
    
    true_grad_lin = np.array( [0] + [1] * n ) 

    true_grad_nonlin = [0] + list( np.cos( np.array( [-1] * n ) ) ) 
    
    rho = np.random.uniform( 0.5 * np.sqrt(n), np.sqrt(2/np.pi) * 1.5 * np.sqrt(n) ) 
    
    g_errors, s_errors, g_vars, b_vars = sphere_res( n+1, rho, mu, true_grad_nonlin, nonlin_func )  
    
    pickle.dump( (g_errors, s_errors, g_vars, b_vars), open( './sphere_nonlin/n={0}_mu={1}'.format(n , mu) , 'wb' ) ) 

10 1 0.01 0.5
10 1 0.001 0.5
20 1 0.01 0.5
20 1 0.001 0.5
30 1 0.01 0.5
30 1 0.001 0.5


# estimation in Euclidean space


In [48]:
def exp_map(v, n ): 
        
    return v

In [49]:
def gaussian_est( func, n, mu = 0.05 , m = 100 , noise_var = 0.0001 ):

    avg_est = 0
    est_var = 0
    
    for _ in range(m):

        v = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
        
        f1 = func( exp_map( mu * v, n), noise_var, n ) 
        f2 = func( exp_map( -mu * v, n), noise_var, n )

        grad_est = (f1 - f2) / ( 2 * mu ) * v 

        avg_est = avg_est + grad_est 
        
        est_var = est_var + np.linalg.norm( grad_est ) ** 4

    avg_est = avg_est / m 
    est_var = est_var / m
    
    return avg_est, est_var

In [50]:
def sphere_est( func, n, alpha, mu = 0.05 , m = 100 , noise_var = 0.0001 ):

    avg_est = 0
    est_var = 0

    for _ in range(m):

        v = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
        v = v/np.linalg.norm(v) * alpha 

        f1 = func( exp_map(mu * v, n), noise_var, n ) 
        f2 = func( exp_map(-mu * v, n), noise_var, n )

        grad_est = n * (f1 - f2) / (2 *  mu ) * v / alpha**2

        avg_est = avg_est + grad_est 
        
        est_var = est_var + np.linalg.norm( grad_est ) ** 4

    avg_est = avg_est / m 
    est_var = est_var / m
    
    return avg_est, est_var

In [51]:
def euc_res(n, alpha, mu, true_grad, f, N = 100 ): 

    g_errors = [] ; g_vars = []

    for _ in range(N): 
        
        avg_est, est_var = gaussian_est(f, n = n, m = SAMPLE_SIZE, mu = mu )
    
        g_errors.append( np.linalg.norm( avg_est - true_grad ) ) 
        
        g_vars.append(est_var) 
    
    s_errors = [] ; s_vars = []

    true_grad_lin = np.array( [0] + [1] * (n-1) )

    for _ in range(N): 
        
        avg_est, est_var = sphere_est(f, n = n, alpha = alpha, m = SAMPLE_SIZE, mu = mu ) 
    
        s_errors.append( np.linalg.norm( avg_est - true_grad ) ) 
        
        s_vars.append(est_var)
    
    return g_errors, s_errors, g_vars, s_vars

In [52]:
ns = [10,20,30] 
alphas = [1]  
betas = [1./2]  
mus = [0.001, 0.01] 

n_alpha_mu_beta = list( itertools.product( ns, alphas, mus, betas ) ) 

res = [] 

for (n, alpha, mu, beta) in n_alpha_mu_beta: 
    
    print(n,alpha,mu,beta)
    
    true_grad_lin = np.array( [1] * n ) 

    true_grad_nonlin = np.cos( np.array( [-1] * n ) ) 
    
    g_errors, s_errors, g_vars, s_vars = euc_res( n, np.sqrt(n), mu, true_grad_lin, lin_func )  
    
    pickle.dump( (g_errors, s_errors, g_vars, s_vars), open( './euc_lin/n={0}_mu={1}'.format(n, mu) , 'wb' ) ) 

10 1 0.001 0.5
10 1 0.01 0.5
20 1 0.001 0.5
20 1 0.01 0.5
30 1 0.001 0.5
30 1 0.01 0.5


In [53]:
ns = [10,20,30] 
alphas = [1]  
betas = [1./2]  
mus = [0.001, 0.01] 

n_alpha_mu_beta = list( itertools.product( ns, alphas, mus, betas ) ) 

res = [] 

for (n, alpha, mu, beta) in n_alpha_mu_beta: 
    
    print(n,alpha,mu,beta)
    
    true_grad_lin = np.array( [1] * n ) 

    true_grad_nonlin = np.cos( np.array( [-1] * n ) ) 
    
    g_errors, s_errors, g_vars, s_vars = euc_res( n, np.sqrt(n), mu, true_grad_nonlin, nonlin_func )  
    
    pickle.dump( (g_errors, s_errors, g_vars, s_vars), open( './euc_nonlin/n={0}_mu={1}'.format(n, mu) , 'wb' ) ) 

10 1 0.001 0.5
10 1 0.01 0.5
20 1 0.001 0.5
20 1 0.01 0.5
30 1 0.001 0.5
30 1 0.01 0.5


# estimation on function surface

In [54]:
def exp_map(v, n ): 
    
    h = np.sqrt( 1 + np.sum(v**4) )/2. + np.arcsinh( np.sqrt(np.sum(v**4)) ) / np.sqrt(np.sum(v**4))
    
    v = np.array( list(v) + [h] )
    
    return v

In [55]:
def gaussian_est( func, n, mu = 0.05 , m = 100 , noise_var = 0.0001 ):

    avg_est = 0 
    est_var = 0
    
    for _ in range(m):

        v = np.random.multivariate_normal(np.zeros(n-1), np.identity(n-1) )
        
        f1 = func( exp_map( mu * v, n), noise_var, n ) 
        f2 = func( exp_map( -mu * v, n), noise_var, n )

        grad_est = (f1 - f2) / ( 2 * mu ) * v 

        avg_est = avg_est + grad_est 
        
        est_var = est_var + np.linalg.norm( grad_est ) ** 4

    avg_est = avg_est / m 
    est_var = est_var / m
    
    return avg_est, est_var

In [56]:
def sphere_est( func, n, alpha, mu = 0.05 , m = 100 , noise_var = 0.0001 ):

    avg_est = 0
    est_var = 0

    for _ in range(m):

        v = np.random.multivariate_normal(np.zeros(n-1), np.identity(n-1) )
        v = v/np.linalg.norm(v) * alpha 

        f1 = func( exp_map(mu * v, n), noise_var, n ) 
        f2 = func( exp_map(-mu * v, n), noise_var, n )

        grad_est = n * (f1 - f2) / (2 *  mu ) * v / alpha**2

        avg_est = avg_est + grad_est 
        
        est_var = est_var + np.linalg.norm( grad_est ) ** 4

    avg_est = avg_est / m 
    est_var = est_var / m
    
    return avg_est, est_var

In [57]:
def surface_res(n, alpha, mu, true_grad, f, N = 100 ): 

    g_errors = [] ; g_vars = []

    for _ in range(N): 
        
        avg_est, est_var = gaussian_est(f, n = n, m = SAMPLE_SIZE, mu = mu )
    
        g_errors.append( np.linalg.norm( avg_est - true_grad ) ) 
        
        g_vars.append(est_var) 
    
    s_errors = [] ; s_vars = []

    true_grad_lin = np.array( [0] + [1] * (n-1) )

    for _ in range(N): 
        
        avg_est, est_var = sphere_est(f, n = n, alpha = alpha, m = SAMPLE_SIZE, mu = mu ) 
    
        s_errors.append( np.linalg.norm( avg_est - true_grad ) ) 
        
        s_vars.append(est_var)
    
    return g_errors, s_errors, g_vars, s_vars

In [58]:
ns = [10,20,30] 
alphas = [1]  
betas = [1./2]  
mus = [0.001, 0.01] 

n_alpha_mu_beta = list( itertools.product( ns, alphas, mus, betas ) ) 

res = [] 

for (n, alpha, mu, beta) in n_alpha_mu_beta: 
    
    print(n,alpha,mu,beta)
    
    true_grad_lin = np.array( [1] * n ) 

    true_grad_nonlin = list( np.cos( np.array( [-1] * n ) ) ) 
    
    g_errors, s_errors, g_vars, s_vars = surface_res( n+1, alpha * n**beta, mu, true_grad_lin, lin_func )  
    
    pickle.dump( (g_errors, s_errors, g_vars, s_vars), 
        open( './surface_lin/n={0}_mu={1}'.format(n , mu) , 'wb' ) ) 

10 1 0.001 0.5
10 1 0.01 0.5
20 1 0.001 0.5
20 1 0.01 0.5
30 1 0.001 0.5
30 1 0.01 0.5


In [59]:
ns = [10,20,30] 
alphas = [1]  
betas = [1./2]  
mus = [0.001, 0.01] 

n_alpha_mu_beta = list( itertools.product( ns, alphas, mus, betas ) ) 

res = [] 

for (n, alpha, mu, beta) in n_alpha_mu_beta: 
    
    print(n,alpha,mu,beta)
    
    true_grad_lin = np.array( [1] * n ) 

    true_grad_nonlin = list( np.cos( np.array( [-1] * n ) ) )
    
    g_errors, s_errors, g_vars, s_vars = surface_res( n+1, alpha * n**beta, mu, true_grad_nonlin, nonlin_func )  
    
    pickle.dump( (g_errors, s_errors, g_vars, s_vars), 
        open( './surface_nonlin/n={0}_mu={1}'.format(n, mu) , 'wb' ) ) 

10 1 0.001 0.5
10 1 0.01 0.5
20 1 0.001 0.5
20 1 0.01 0.5
30 1 0.001 0.5
30 1 0.01 0.5
